PRIVACY PRESERVING AI - DP (Adding noise to the database)

In [ ]:
import torch

# the number of entries in our database
num_entries = 5000

db = torch.rand(num_entries) > 0.5

In [ ]:
db.shape

torch.Size([5000])

In [ ]:
db

tensor([ True, False,  True,  ...,  True,  True, False])

In [ ]:
def get_parallel_db(db, remove_index):

    return torch.cat((db[0:remove_index], 
                      db[remove_index+1:]))

In [ ]:
get_parallel_db(db, 1)

tensor([ True,  True, False,  ...,  True,  True, False])

In [ ]:
def get_parallel_dbs(db):

    parallel_dbs = list()

    for i in range(len(db)):
        pdb = get_parallel_db(db, i)
        parallel_dbs.append(pdb)
    
    return parallel_dbs

In [ ]:
pdbs = get_parallel_dbs(db)

In [ ]:
def create_db_and_parallels(num_entries):
    
    db = torch.rand(num_entries) > 0.5
    pdbs = get_parallel_dbs(db)
    
    return db, pdbs

In [ ]:
db, pdbs = create_db_and_parallels(20)

In [ ]:
db.float()

tensor([1., 1., 0., 1., 0., 0., 1., 0., 0., 1., 1., 1., 1., 0., 0., 0., 0., 1.,
        1., 1.])

In [ ]:
#Local Differential Privacy

In [ ]:
def query(db):

    true_result = torch.mean(db.float())
    
    first_coin_flip = (torch.rand(len(db)) > 0.5).float()
    second_coin_flip = (torch.rand(len(db)) > 0.5).float()

    augmented_database = db.float() * first_coin_flip + (1 - first_coin_flip) * second_coin_flip

    db_result = torch.mean(augmented_database.float()) * 2 - 0.5
    
    return db_result, true_result

In [ ]:
(torch.rand(len(db)) > 0.5).float()

tensor([1., 1., 1., 0., 1., 0., 1., 1., 0., 0.])

In [ ]:
db, pdbs = create_db_and_parallels(10)
private_result, true_result = query(db)
print("With Noise:" + str(private_result))
print("Without Noise:" + str(true_result))

With Noise:tensor(0.7000)
Without Noise:tensor(0.4000)


In [ ]:
db, pdbs = create_db_and_parallels(10000)
private_result, true_result = query(db)
print("With Noise:" + str(private_result))
print("Without Noise:" + str(true_result))

With Noise:tensor(0.5006)
Without Noise:tensor(0.4981)


In [ ]:
# Creating a Differentially private query

In [ ]:
epsilon = 0.0001

In [ ]:
import numpy as np

In [ ]:
db, pdbs = create_db_and_parallels(100)

In [ ]:
def sum_query(db):
    return db.sum()

In [ ]:
def mean_query(db):
    return torch.mean(db.float())

In [ ]:
def laplacian_mechanism(db, query, sensitivity):
    
    beta = sensitivity / epsilon
    noise = torch.tensor(np.random.laplace(0, beta, 1))
    
    return query(db) + noise

In [ ]:
laplacian_mechanism(db, sum_query, 1)

tensor([8618.6069], dtype=torch.float64)

In [ ]:
laplacian_mechanism(db, mean_query, 1/100)

tensor([55.6272], dtype=torch.float64)